# Nuclio - Grafana's World Map Integration

In [1]:
# nuclio: ignore
import nuclio

## Environment & Definitions

In [2]:
%nuclio env V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable


In [3]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


In [4]:
%nuclio config spec.triggers.httpEndpoint.kind = "http"
%nuclio config spec.triggers.httpEndpoint.attributes.port = 32012

%nuclio: setting spec.triggers.httpEndpoint.kind to 'http'
%nuclio: setting spec.triggers.httpEndpoint.attributes.port to 32012


In [5]:
%nuclio cmd -c pip install pandas
%nuclio cmd -c pip install v3io_frames

## Function

In [6]:
import v3io_frames as v3f

In [7]:
def init_context(context):
    client = v3f.Client('framesd:8081', container='bigdata')
    setattr(context, 'v3f', client)

In [8]:
def handler(context, event):
    df = context.v3f.read('kv', 'stores'). \
        loc[:, ['name', 'longitude', 'latitude', 'customers']]
    df['key'] = df['name']
    return df.to_json(orient='records')

# Test

In [9]:
# nuclio: ignore
init_context(context)
event = nuclio.Event(body='')
out = handler(context, event)
out

WriteError: error in _read: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "can't query: Expected a 2xx response status code: HTTP/1.1 404 Not Found
Server: nginx
Date: Thu, 01 Aug 2019 07:36:34 GMT
Content-Type: application/json
Content-Length: 66
Connection: keep-alive

{
	"ErrorCode": -2,
	"ErrorMessage": "No such file or directory"
}"
	debug_error_string = "{"created":"@1564644994.858744262","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"can't query: Expected a 2xx response status code: HTTP/1.1 404 Not Found\r\nServer: nginx\r\nDate: Thu, 01 Aug 2019 07:36:34 GMT\r\nContent-Type: application/json\r\nContent-Length: 66\r\nConnection: keep-alive\r\n\r\n{\n\t"ErrorCode": -2,\n\t"ErrorMessage": "No such file or directory"\n}","grpc_status":2}"
>

# Deploy

In [9]:
%nuclio deploy -n worldmap-endpoint -p recommendation_engine -c

[nuclio.deploy] 2019-08-01 07:40:25,640 project name not found created new (recommendation_engine)
[nuclio.deploy] 2019-08-01 07:40:26,732 (info) Building processor image
[nuclio.deploy] 2019-08-01 07:40:54,042 (info) Pushing image
[nuclio.deploy] 2019-08-01 07:41:06,240 (info) Build complete
[nuclio.deploy] 2019-08-01 07:41:10,301 (info) Function deploy complete
[nuclio.deploy] 2019-08-01 07:41:10,314 done creating worldmap-endpoint, function address: 3.120.15.118:32012
%nuclio: function deployed
